# NOSQL AND QUERYING JSON AND XML/HTML OBJECTS/FILES

## Working with JSON Objects

- JSON is a hierarchical data format that allows data that is not appropriate to be formatted as columns and rows to be stored and queried.

- Let's say we are tracking our contact data in a csv file:

```
Lastname, Firstname, Phone Number

Membrey, Peter, +852 1234 5678

Thielen, Wouter, +81 1234 5678
```

- If one of the contacts have more than one phone numbers, we have to create a new column

```
Lastname, Firstname, Phone Number1, Phone Number2

Membrey, Peter, +852 1234 5678, +44 1234 565 555

Thielen, Wouter, +81 1234 5678
```

- But suppose, we have million of records with tens of fields, and in some exceptions, some records may have many multiple values of some fields: e.g. 10 telephone numbers, etc

- JSON format is a remedy for these kinds of flexibility issues and hierarchical data formats.

- Integrity rules are softer for handling JSON data

- JSON stands for "JavaScript Object Notation"
- In JSON, each record is called a "document"
- Let's write the first record as a JSON document:

In [ ]:
record1='{
"firstname": "Peter",
"lastname": "Membrey",
"phone_numbers": [
"+852 1234 5678",
"+44 1234 565 555"
]
}'

echo $record1

- Echoing the JSON as comma separated values as such, is not easy to parse and understand the format.

- We may use online json parsers for this purpose.

- You can copy and paste the above string into the input pane:

http://jsonparseronline.com/

### Querying JSON with JQ

- Or better, we will use a handy tool called "jq" for this purpose:

In [ ]:
echo $record1 | jq .

jq is a parser and querying tool for json, that creates a nice output

You can have more info on jq following the links:

[The Home Page](https://stedolan.github.io/jq/)

[Tutorial](https://stedolan.github.io/jq/tutorial/)

[Manual](https://stedolan.github.io/jq/manual/)

- Each document (equivalent to a row in RDBMS) in JSON is delimited by curly braces "{"
- And all values are given as "key" and "value" pairs:

```json
{
  "firstname": "Peter",
  "lastname": "Membrey",
  "phone_numbers": [
    "+852 1234 5678",
    "+44 1234 565 555"
  ]
}
```

- firstname is the key, "Peter" is the value, an so on

- We also have arrays of values for a single key, delimited by square brackets []

- What is more powerful about JSON format is that you can embedded documents inside other ones: (we print the data here in three visual formats)

In [ ]:
record1b='{
"firstname": "Peter",
"lastname": "Membrey",
"numbers": [
{
"phone": "+852 1234 5678"
},
{
"fax": "+44 1234 565 555"
}
]
}'

echo -e $record1b "\n"

echo -e "$record1b\n"

echo $record1b | jq .

- See, the phone and fax numbers are inside embedded documents

- Multiple documents can be "collected" inside "collections":

- A "collection" in NoSQL terminology is analogous to a "table" in the RDBMS jargon. A collection is a collection of similar items (or documents with similar key-value pairs)

### A real json database example: UN COMTRADE

- We will be using a part of the UN COMTRADE database:

[UN COMTRADE](https://comtrade.un.org/)

UN COMTRADE is the widest and most comprehensive database on international trade:

- 250+ reporter countries
- 290+ partner countries
- 6500+ commodity codes
- 50+ of history
- Both imports and exports
- Both values and quantities!

Let's first save the path:

In [ ]:
datadir=~/data
comtrade=$datadir/comtrade_s1

In [ ]:
ls $comtrade

- classificationS1.json lists the item classification according to SITC1 method
- reporterAreas.json and partnerAreas.json lists the countries and their respective codes
- data files are under 2010

#### Reporters

In [ ]:
jq . $comtrade/reporterAreas.json

Another way to pretty print and navigate through json files is R:

In [ ]:
reporter <- jsonlite::fromJSON("~/data/comtrade_s1/reporterAreas.json")
reporter

In [ ]:
str(reporter)

See that, in not-so-nested structures, the data is automatically flattened into a data frame

And a collapsable and interactive gadget for viewing json and similar hierarchical data types: (you may need two execute several times to get the JS gadget)

In [ ]:
listviewer::jsonedit(reporter, mode = "form")

Now let's traverse through this document to list country texts:

In [ ]:
jq '.results[].text' $comtrade/reporterAreas.json | tr -d '"'

And let's list the country codes:

In [ ]:
jq -r '.results[].id' $comtrade/reporterAreas.json

- Separate lists of country names and id's do not mean much.
- Suppose we want to find the country code of turkey

In [ ]:
reporters=$(jq -r '.results[] | "\(.text)\t\(.id)"' $comtrade/reporterAreas.json | \
xargs -0 -i echo -e "{}")

echo "$reporters" |  column -s $'\t' -t

We can filter with grep or awk:

In [ ]:
echo "$reporters" | awk -F "\t" '$1=="Turkey"{print $2}'

Another way to flatten fields is:

In [ ]:
reporters2=$(jq -r '.results[] | .text + "\t" + .id' $comtrade/reporterAreas.json | xargs -0 -i echo -e "{}")

echo "$reporters2" |  column -s $'\t' -t

In [ ]:
echo "$reporters" | awk -F "\t" '$1=="Turkey"{print $2}'

Or we can use the "select" statement for filtering values:

In [ ]:
jq -r '.results[] | select(.text == "Turkey") | .id' $comtrade/reporterAreas.json

- All the files under 2010e directory are files in which Turkey is either a reporter or a partner
- So sometimes there exists a pair of files where Turkey and another country swap roles as reporter and partner

#### Classification

Now let's go through the classification file:

In [ ]:
classification <- jsonlite::fromJSON("~/data/comtrade_s1/classificationS1.json")
listviewer::jsonedit(classification, mode = "form")

Now we will filter for those entries, in which text includes "textile" and code is only 3 digits:

In [ ]:
jq -r '.results[] | select((.id|test("^\\d+$")) and (.text|test("(?i)textile"))) |
select((.id|tonumber < 1000) and (.id|tonumber > 99)) | .text' \
$comtrade/classificationS1.json | \
sed 's/ - /\t/g' | column -s $'\t' -t

See how it works:

- We filter for id values that are numeric (so exclude ALL, TOTAL, AG1..AG5) and where text includes case insensitive textile
- We filter for id values larger than 99 and smaller than 1000
- We return the text
- The text has already id info at the beginning split with " - ". We substitute these character with a tab character

**EXERCISE 1:**

Now your turn: Find the id and text of codes parent of which are 651 (Textile yarn and thread).

The result will be:

```
6511  Thrown silk & silk yarn and thread
6512  Yarn of wool and animal hair
6513  Cotton yarn & thread, grey, not mercerized
6514  Cotton yarn & thread, bleached, dyed, mercerd.
6515  Yarn and thread of flax, ramie and true hemp
6516  Yarn and thread of synthetic fibres
6517  Yarn and thread of regenerated fibres
6518  Yarn of glass fibre
6519  Yarn of textile fibres,nes incl.paper yarn
```

**SOLUTION 1:**

In [ ]:
pass1=
encrypt="U2FsdGVkX1+WtEYqehLba8zmJ9evvmr1ohvj0BWaggG2cQUjY/HMZia25jIkV9zR jyLxrSmHJRCX55wFh7JIB9U1sjWJ1cvipesjhECBk3B3GaP97Kjsq9cDu0jnjYu7 cM9b9CKPMJN1Am/sywL5RQT+z0nDKopIuq78l3p+eeqEHQNvloe35hu/y19+lFmt 6f4EG2i1cQ2JJq8bBMyl9g=="
solution=$(echo "$encrypt" | openssl enc -aes-128-cbc -a -d -salt -pass pass:$pass1 2> /dev/null)
echo "$solution"; echo
for l in "$solution"; do eval "${l}"; done

#### Data files

Now let's go through the actual data files that includes trade volumes:

In [ ]:
ls $comtrade/2010e | { head -5; tail -5; }

In [ ]:
ls $comtrade/2010e | grep -P "2010_792" | head -5

- The code after the first underscore is the reporter's and the code after the second underscore is the partner country's code

- So there are the files for which Turkey is either a reporter or partner

- For how many files Turkey is reporter or a partner?

In [ ]:
ls $comtrade/2010e | grep -P "2010_792" | wc -l
ls $comtrade/2010e | grep -P "792_s1" | wc -l

Turkey reported her trade with 227 partners, while 165 reporter countries reported their trade partnering with Turkey (for s1 classification only)

In [ ]:
tradedata <- jsonlite::fromJSON("~/data/comtrade_s1/2010e/2010_792_100_s1.json")
listviewer::jsonedit(tradedata, mode = "form")

Descriptions for several variables are:

- CmdCode: 	Commodity code
- CmdDesc: 	Commodity description
- IsLeaf: 	Basic code without children
- Parentcode: 	High level of that commodity code
- pfDesc: 	Commodity classification
- PfCode: 	Commodity classification code
- yr: 	Year
- rtCode 	Reporter Code
- ptCode: 	Partner Code
- qtCode: 	Quantity code 

Now from all files for which Turkey is a reporter, the TradeValue of exports (rgCode is 2) in 651 code (Textile yarn and thread)

We will report:
- ptTitle (name of partner country)
- TradeValue
- TradeQuantity 

For a single file we have:

In [ ]:
jq -r '.dataset[] | select(.cmdCode == "651" and .rgCode == 2) |
"\(.ptTitle)\t\(.TradeValue)\t\(.TradeQuantity)"' $comtrade/2010e/2010_792_100_s1.json | \
xargs -0 -i echo -e "{}"

In [ ]:
textileexports=$(for file in $comtrade/2010e/2010_792*.json;
do
    jq -r '.dataset[] | select(.cmdCode == "651" and .rgCode == 2) |
"\(.ptTitle)\t\(.TradeValue)\t\(.TradeQuantity)"' $file | \
xargs -0 -i echo -e "{}";
done)

In [ ]:
echo "$textileexports" | column -s $'\t' -t

Let's exclude World total:

In [ ]:
echo "$textileexports" | awk -F "\t" '$1 != "World"' | \
column -s $'\t' -t

In [ ]:
echo "$textileexports" | \
awk -F "\t" '$1 != "World"' | \
sort -t $'\t' -nr -k 2 | \
numfmt -d $'\t' --field=2-3 --to-unit=M | \
column -s $'\t' -t

What we do here:

- Exclude the row first field of which had "World"
- Sort as number in reverse order by second field
- Format 2nd-3rd columns in millions
- And align columns

So in 2010, Turkey's significant partners for Textile yarn and thread export are:

- Italy (USD 166 mio)
- Russion Federation (USD 95 mio)
- Germany (USD 86 mio)

**EXERCISE 2:**

- For each file in which Turkey is a partner, return the reporting country name (rtTitle), comodity description (cmdDescE) and trade value (TradeValue) for those items which:
- are exported from those countries to Turkey (rgCode is 2)
- TradeValue's are at least 500000000 (500 mio)
- and are at the most detailed level (IsLeaf is 1) 

So you should fill in this template below:

```Bash
for file in $comtrade/2010e/*792_s1.json;
do
    jq -r '#fill in this part' $file | \
    xargs -0 -i echo -e "{}";
done | \
awk -F "\t" '$1 != "EU-28"' | \
sort -t $'\t' -nr -k 3 | \
numfmt -d $'\t' --field=3 --to-unit=M | \
column -s $'\t' -t
```

The outpu will be:

```
Germany             Passenger motor cars, other than buses           2653
Algeria             Gas, natural                                     2591
USA                 Special transactions                             2010
Russian Federation  Crude petroleum                                  1607
Germany             Other parts for motor vehicles                   1231
China               Statistical machines cards or tapes              1027
France              Other parts for motor vehicles                   1003
Germany             Internal combustion engines, not for aircraft    985
Russian Federation  Coal /anthracite, bituminous/                    972
Germany             Special transactions                             948
Spain               Passenger motor cars, other than buses           923
Russian Federation  Aluminium and aluminium alloys, unwrought        865
USA                 Raw cotton, other than linters                   862
Russian Federation  Refined copper including remelted                861
Saudi Arabia        Products of polymerization and copolymerization  849
Italy               Other parts for motor vehicles                   806
France              Aircraft, heavier than air                       799
Belgium             Products of polymerization and copolymerization  644
Qatar               Gas, natural                                     631
Italy               Machinery and mechanical appliances, nes         562
```

You can view a sample data:

In [ ]:
tradedata <- jsonlite::fromJSON("~/data/comtrade_s1/2010e/2010_100_792_s1.json")
listviewer::jsonedit(tradedata, mode = "form")

**SOLUTION 2:**

In [ ]:
pass1=
encrypt="U2FsdGVkX1+4esKDuc9iGGz0tWhqscfUMmjPGTa2MIp+YPGWOvdJZY0ZLMx3XSI5 42ueQ3AERCACRgSCZOnUhnDJG4wgnAgcmv+RXW0fR82yLC6g5M2+t3XuODqTIZag Y+g6DkbnapcedfV2K5hg0ELct8YzomN7648I/vQxCQEYEPlwxv/Pdw7IBEeIgFTH zJ25jJlAKJerLYTl9G+XXr01Pwlh5VS36wh16/opSTYjeh++FgvCCH2WKBiFfLjK oxzjfLBcpcqWLVOQ8ozqCean7tnTPIHR0QzjJeiqJ91jBOLbxPpnyT93Yu5Ikxne MbZ/7zAVqLmQVJCpLj6igmHtYxMv3Ck1HC5RKLX2Qrb8sPygcafwyGRp+r0jEsOk nl3MxKOVMuWIwvA+gmJwQhUJXmf7Zm1YX9tcn3PJe6IWPom2P9esB+GXKMnFkpL9 IsDEP/nvQ5TXe+gDGpKWzlEirPp6CwAB6YNiH/phRMUmrbg2DtsRSvZA9bMslFbf eXrItWZ34YuwnqDSlAaquA=="
solution=$(echo "$encrypt" | openssl enc -aes-128-cbc -a -d -salt -pass pass:$pass1 2> /dev/null)
echo "$solution"; echo
for l in "$solution"; do eval "${l}"; done

So according to numbers reported by other countries, in 2010 Turkey's largest imports were:

- Passenger cars, engines and parts for motor vehicles from Germany and France (some portion for domestic production)
- Natural gas, petroleum and Coal from Algeria and Russia
- (Probably) mobile devices from China

Note that SITC 1 is a very old classification so some modern commodities may not fit well into this schema

### MongoDB

Now we will import the files into MongoDB

First we create a directory for MongoDB database:

In [ ]:
mkdir -p ~/mongo

And run the mongo server:

In [ ]:
mongod -dbpath ~/mongo &

And import the files into a new database called comtrade and collection called 2010s1:

**DO NOT REPEAT THIS STEP MULTIPLE TIMES, YOU WILL HAVE DUPLICATE DOCUMENTS!**

In [ ]:
for file in $comtrade/2010e/*.json;
do mongoimport --db comtrade --collection 2010s1 $file; done

Now from R, import monglite package: 

In [ ]:
library(mongolite)

We create a connection object:

In [ ]:
con <- mongo(collection = "2010s1", db = "comtrade")

Select the document(s) where rtTitle is Bulgaria and return, commodity description, commodity code and TradeValue columns:

In [ ]:
bulgaria_vals <- con$find('{"dataset.rtTitle" : "Bulgaria"}',
               fields = '{"dataset.cmdDescE": 1, "dataset.cmdCode": 1, "dataset.TradeValue":1 }')

The syntax of find() method has
- The query
- And the projection (selected fields to be returned)

The fields to be returned get the value 1

Let's view the structure of the return value:

In [ ]:
str(bulgaria_vals)

And view only the dataset:

In [ ]:
bulgaria_vals$dataset

Now we will find those partner countries of Turkey with which Turkey made exports of more than USD 10 bio in 2010: 

In [ ]:
sapply(large_trade$dataset, function(x) x$ptTitle[1])

In [ ]:
export_651 <- con$find('{
"dataset":
{ "$elemMatch":

{ "$and": [
    { "rtTitle" : "Turkey" },
    { "cmdCode" : "651" },
    { "TradeValue" : { "$gt": 50000000 } },
    { "ptTitle" : { "$ne" : "World" } },
    { "rgCode" : 2 }

] } } }',
               fields = '{"dataset.ptTitle": 1,
                            "dataset.rgCode": 1,
                            "dataset.cmdDescE": 1,
                            "dataset.cmdCode": 1,
                            "dataset.TradeValue":1 }')

In [ ]:
str(export_651)

In [ ]:
export_651_2 <- do.call(rbind, export_651$dataset)
with(export_651_2, export_651_2[rgCode == "2" & cmdCode == "651",]) 

**EXERCISE 3:**

Select those countries for which Turkey has at least USD 5 bio exports

**SOLUTION 3:**

In [ ]:
pass <- readline(prompt = "Please enter the password for the solution: ")
encrypt <- "U2FsdGVkX1/cPhgJB5HjYQ4xWOnuyfs5vKrofNpNlV9UHETg+d+KYoLJw7Py0t9p ap3m5rxDCl7JtmwH6VZKMf5sfooPDhhAnI5YXHJJmAc78U5ZYT/aVL75466bi6Te ic4Yd4uM1jcyblnj9LAp2n76eyGFFaEmAY2hmwn+uaDbk91xTNTYnE0gNqmjMJHs t60XDU19QfOTF8YUyG1cbYQwGqMp4k9u1o45Co5f4BRH8aWrgdCp1NBCd+X2itsl ujQYQ45+dnj7D01iezb9gPxvWiEMT+rIP7rWEQh8+wg/mIRR0zctqwk0xrIXLSNo /q1RCrw8DTaoqX5De3EZXxFzWNsw81+/9eehspVmosb9KnoNEKfZzgj99dm/ICDq 8uf9bqxZ373tN6oC23znD1vndhDPVl9KbwoNHjLa9Cb+Do5qoQqQ+hxEnDQm9I96 jai0dRPTnf5RXeb4Zs4Us3+8wahR3gm797WU/qJVIymbZH+5DjSID27XCI2WinPi"
solution <- system(sprintf("echo %s | openssl enc -md sha256 -aes-128-cbc -a -d -salt -pass pass:%s 2> /dev/null", encrypt, pass), intern = T, ignore.stderr = T)
cat(solution, sep = "\n")
eval(parse(text = solution))

## Working with XML/HTML Objects